# Projeto 1 - Ciência dos Dados

Nome: Ralph Michaan

Nome: Fábio Ferreira

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'iPhone.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPhone.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename, sheet_name = 'Treinamento')
train.head(5)

,Treinamento,Classificação
0,feliz pelo iphone 11 mas o iphone 10 mds é mui...,0
1,quando eu estiver comprando o iphone 11 pro ma...,1
2,"não consegui comprar o iphone 11,imagina o 14",1
3,graças a deus vou compra meu iphone 11 pro max 🤩,1
4,"gente, qual a diferença pro iphone 11? kkkkkkk...",1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,@hyzerfps iphone 11 = fogao de 3 bocas\niphone...,0
1,@sirmaykon @uourah kkkkkkkkkkkkkkkkkkkkkk tem ...,0
2,@cnnbrbusiness @cnnbrasil por isso o mundo tá ...,0
3,rt @k3viyn: #appleevent esperando o iphone 12 ...,0
4,"é meus amigos, que fase! troquei gramado por u...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é um smartphone da Apple, lançado em setembro de 2019, por um preço um pouco mais baixo que suas variantes mais parrudas.

Consideramos relevante os tweets em que havia algum feedback sobre o produto, ou se a pessoa tinha vontade de ter, já os desconsiderados foram as piadas feitas com o valor, e os tweets que não se tratavam do iPhone 11 propriamente dito.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

Limpando os tweets e pegando as frequências absolutas e relativas:

In [5]:
import re

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

Dos tweets relevantes:

In [6]:
train_relevante_clean = train.loc[train["Classificação"] == 1, 'Treinamento'].apply(cleanup)
train_relevante_clean

1      quando eu estiver comprando o iphone 11 pro ma...
2           não consegui comprar o iphone 11imagina o 14
3       graças a deus vou compra meu iphone 11 pro max 🤩
4      gente qual a diferença pro iphone 11 kkkkkkkk ...
7      @ginghinni7 @conservadorarv melhor do que comp...
                             ...                        
492    rt @juh_santos14 troco minha amiga por um ipho...
493          @rodsal200 um iphone 11 tá mais cara q isso
494    contando os dias pra pegar meu iphone 11😍😍😍😍pr...
495    rt @vick_417 e eu que quase comprei o iphone 1...
498    @wulfs1ghs ᅟᅟᅟ   ⠀\n/entrega uma caixa do ipho...
Name: Treinamento, Length: 196, dtype: object

In [30]:
train_relevante_clean.astype("string")
train_relevante_lower = train_relevante_clean.str.lower()
train_relevante = train_relevante_lower.str.split()


train_relevante_lista = []

for palavras in train_relevante:
    train_relevante_lista += palavras
serie_train_relevante = pd.Series(train_relevante_lista)
serie_train_relevante

0          quando
1              eu
2         estiver
3       comprando
4               o
          ...    
3434           um
3435          ceo
3436        legal
3437          ᅟᅟᅟ
3438            ⠀
Length: 3439, dtype: object

In [9]:
serie_train_relevante_absoluta = serie_train_relevante.value_counts()
serie_train_relevante_absoluta

iphone          216
11              194
o               159
um               92
que              72
               ... 
@whindersson      1
solo              1
estressa          1
inventam          1
querem            1
Length: 1065, dtype: int64

In [11]:
serie_train_relevante_relativa = serie_train_relevante.value_counts(True)
serie_train_relevante_relativa

iphone          0.062809
11              0.056412
o               0.046234
um              0.026752
que             0.020936
                  ...   
@whindersson    0.000291
solo            0.000291
estressa        0.000291
inventam        0.000291
querem          0.000291
Length: 1065, dtype: float64

E dos irrelevantes:

In [12]:
train_irrelevante_clean = train.loc[train["Classificação"] == 0, 'Treinamento'].apply(cleanup)
train_irrelevante_clean

0      feliz pelo iphone 11 mas o iphone 10 mds é mui...
5      fui aí à dois anos quando ofereci um iphone 11...
6      gosto muito do conceito do pobre com macbook p...
9      @gab_neres ce pedia meu rg pra ir no motel e e...
10     apareceu uma msg pro luan no google dizendo qu...
                             ...                        
490    eu só preciso saber quanto vai valer o iphone ...
491    um apoiador das causas da comunidade lgbtq pra...
496    e nem vou comentar o “comunistas” e o “anti ca...
497    rt @rannasoareez comprei o iphone 11 e aí inve...
499    preciso de um telefone novo alguém me dá um ip...
Name: Treinamento, Length: 304, dtype: object

In [31]:
train_irrelevante_clean.astype("string")
train_irrelevante_lower = train_irrelevante_clean.str.lower()
train_irrelevante = train_irrelevante_lower.str.split()

train_irrelevante_lista = []

for palavras in train_irrelevante:
    train_irrelevante_lista += palavras
serie_train_irrelevante = pd.Series(train_irrelevante_lista)
serie_train_irrelevante

0        feliz
1         pelo
2       iphone
3           11
4          mas
         ...  
5969        um
5970    iphone
5971        11
5972       por
5973     favor
Length: 5974, dtype: object

In [14]:
train_irrelevante_absoluta = serie_train_irrelevante.value_counts()
train_irrelevante_absoluta

iphone                   354
11                       302
o                        201
um                       166
de                       144
                        ... 
gastei                     1
bglho                      1
sul                        1
https//tco/au5liicjhm      1
travando                   1
Length: 1750, dtype: int64

In [15]:
train_irrelevante_relativa = serie_train_irrelevante.value_counts(True)
train_irrelevante_relativa

iphone                   0.059257
11                       0.050552
o                        0.033646
um                       0.027787
de                       0.024104
                           ...   
gastei                   0.000167
bglho                    0.000167
sul                      0.000167
https//tco/au5liicjhm    0.000167
travando                 0.000167
Length: 1750, dtype: float64

Criando a lista de palavras dos tweets e fazendo o smoothing:

In [43]:
portugues = train_relevante_lista + train_irrelevante_lista
qntd_port = len(portugues)
qntd_port

9413

In [18]:
serie_portugues = pd.Series(portugues)
portugues_relativa = serie_portugues.value_counts(True)
portugues_relativa

iphone              0.060555
11                  0.052693
o                   0.038245
um                  0.027409
de                  0.021460
                      ...   
@lulucosttaa        0.000106
@gabrieldorileo_    0.000106
tacar               0.000106
vacilo              0.000106
🤟🏼                  0.000106
Length: 2298, dtype: float64

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

Limpando e pegando as frequências relativas e absolutas dos tweets relevantes:

In [34]:
teste_relevante_clean = test.loc[test["Relevância"] == 1, 'Teste'].apply(cleanup)
teste_relevante_clean

4      é meus amigos que fase troquei gramado por um ...
5      meu pai comprou um iphone 11 e eu ainda achei ...
7                  vou pegar o iphone 11 final do ano 😍😍
10     a apple já criou o iphone 12 enquanto eu acabe...
12     eu babava pelo carregador turbo do motorola ma...
                             ...                        
223    rt @fernandavsntss eu to muito triste e soment...
225      deixem me ter iphone 11 vão ver só snaps a cair
229    @renatinhapoqui n consigo fazer meu ead pq meu...
234    bateria do iphone 11 é tudooo ainda nem carreg...
237                         @_ilipe quero um iphone 11 🤔
Name: Teste, Length: 89, dtype: object

In [35]:
teste_relevante_clean.astype("string")
teste_relevante_lower = teste_relevante_clean.str.lower()
teste_relevante = teste_relevante_lower.str.split()


teste_relevante_lista = []

for palavras in teste_relevante:
    teste_relevante_lista += palavras
serie_teste_relevante = pd.Series(teste_relevante_lista)
serie_teste_relevante

0            é
1         meus
2       amigos
3          que
4         fase
         ...  
1445     quero
1446        um
1447    iphone
1448        11
1449         🤔
Length: 1450, dtype: object

In [36]:
serie_teste_relevante_absoluta = serie_teste_relevante.value_counts()
serie_teste_relevante_absoluta

iphone     92
11         90
o          67
um         47
eu         30
           ..
s20         1
🤡🤡          1
aquele      1
sistema     1
deixem      1
Length: 547, dtype: int64

In [37]:
serie_teste_relevante_relativa = serie_train_relevante.value_counts(True)
serie_teste_relevante_relativa

iphone          0.062809
11              0.056412
o               0.046234
um              0.026752
que             0.020936
                  ...   
@whindersson    0.000291
solo            0.000291
estressa        0.000291
inventam        0.000291
querem          0.000291
Length: 1065, dtype: float64

E dos irrelevantes:

In [38]:
teste_irrelevante_clean = test.loc[test["Relevância"] == 0, 'Teste'].apply(cleanup)
teste_irrelevante_clean

0      @hyzerfps iphone 11 = fogao de 3 bocas\niphone...
1      @sirmaykon @uourah kkkkkkkkkkkkkkkkkkkkkk tem ...
2      @cnnbrbusiness @cnnbrasil por isso o mundo tá ...
3      rt @k3viyn appleevent esperando o iphone 12 la...
6                                  era só um iphone 11 🤧
                             ...                        
231    eu nunca me esqueço do dia q fui tapeada num s...
232            ganhar um iphone 11 https//tco/coxrl1zkza
233    rezando pra que tenha chegado película pro iph...
235    o candidato de igt bancando o populista matuto...
236    @junior_onda vamos pegar o iphone 11 do nosso ...
Name: Teste, Length: 149, dtype: object

In [39]:
teste_irrelevante_clean.astype("string")
teste_irrelevante_lower = teste_irrelevante_clean.str.lower()
teste_irrelevante = teste_irrelevante_lower.str.split()


teste_irrelevante_lista = []

for palavras in teste_irrelevante:
    teste_irrelevante_lista += palavras
serie_teste_irrelevante = pd.Series(teste_irrelevante_lista)
serie_teste_irrelevante

0                 @hyzerfps
1                    iphone
2                        11
3                         =
4                     fogao
               ...         
2762                     do
2763                  nosso
2764               parceiro
2765       @brendonricardo2
2766    kkkkkkkkkkkkkkkkkkk
Length: 2767, dtype: object

In [40]:
serie_teste_irrelevante_absoluta = serie_teste_irrelevante.value_counts()
serie_teste_irrelevante_absoluta

iphone            167
11                148
o                  91
e                  57
que                57
                 ... 
vendo               1
mac/ipad/watch      1
20                  1
basta               1
f800                1
Length: 964, dtype: int64

In [41]:
serie_teste_irrelevante_relativa = serie_train_irrelevante.value_counts(True)
serie_teste_irrelevante_relativa

iphone                   0.059257
11                       0.050552
o                        0.033646
um                       0.027787
de                       0.024104
                           ...   
gastei                   0.000167
bglho                    0.000167
sul                      0.000167
https//tco/au5liicjhm    0.000167
travando                 0.000167
Length: 1750, dtype: float64

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**